In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import sys
__file__ = os.path.abspath('')
dir_path = '/'.join(os.path.realpath(__file__).split('/')[:-1])
sys.path.append(f'{dir_path}/sabatinilab-glm/backend')
sys.path.append(f'{dir_path}/..')
sys.path.append(f'{dir_path}/backend')
sys.path.append(f'{dir_path}/../backend')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GroupShuffleSplit

import sglm
import sglm_cv
import sglm_pp


In [2]:

df = pd.read_csv('/Users/josh/Documents/Harvard/GLM/C39v2_sampleDesignMat.csv').drop('Unnamed: 0', axis=1).drop('index', axis=1)
print('Data Loaded...')
y_setup_col = 'grnL' # photometry response
df['grnL_diff'] = sglm_pp.diff(df['grnL'])
print('NP Diff Completed...')

X_cols = [
    'nTrial', # trial ID
    # 'iBlock', # block number within session
    'CuePenalty', # lick during cue period (no directionality yet, so binary 0,1)
    'ENLPenalty', # lick during ENL period (no directionality yet, 0,1)
    'Select', # binary selection lick
    'Consumption', # consumption period (from task perspective)
    # 'TO', # timeout trial
    'responseTime', # task state cue to selection window
    'ENL', # task state ENL window
    'Cue', # task state Cue window
    'decision', # choice lick direction (aligned to select but with directionality -1,1)
    'switch', # switch from previous choice on selection (-1,1)
    'selR', # select reward (-1,1) aligned to selection
    'selHigh', # select higher probability port (-1,1)
    'Reward', # reward vs no reward during consumption period (-1,1)
    'post', # log-odds probability
]

y_col = 'grnL_diff'

dfrel = df[X_cols + [y_col]].copy()
dfrel = dfrel.replace('False', 0).astype(float)
dfrel = dfrel*1

X_setup = dfrel[X_cols]
y_setup = dfrel[y_col]

ts = 2

shift_amt_list = [0]
shift_amt_list += list(range(-ts, 0))
shift_amt_list += list(range(1, ts+1))

dfrel = sglm_pp.timeshift_multiple(X_setup, shift_amt_list=shift_amt_list)

print('Timeshift Multiple Done...')

full_dataset = dfrel.copy()
full_dataset['grnL_diff'] = y_setup
full_dataset['grnL_sft'] = y_setup.shift(1)
full_dataset['grnL_sft2'] = y_setup.shift(2)
full_dataset = full_dataset.iloc[5:]
full_dataset = full_dataset.dropna().copy()

X = full_dataset.drop(y_col, axis=1)
y = full_dataset[y_col]

print('SKGLM Starting...')


Data Loaded...
NP Diff Completed...
Timeshift Multiple Done...
SKGLM Starting...


In [7]:
# predictor_size = 16
predictor_size = len(X.columns)
for fc in range(len(X.columns)//predictor_size):
    use_cols = list(range(fc * predictor_size, fc * predictor_size + predictor_size))

    use_cols = [_ for _ in use_cols if 'nTrial' not in X.columns[_]]

    # print(list(X.columns))
    column_names = X.columns[use_cols]
    print(f"{use_cols} — {column_names}")
    # glm3 = sglm.SKGLM('Normal', max_iter=10000, alpha=0)
    # glm3.fit(X.values[1:, use_cols], y.values[1:])
    
    glm4 = LinearRegression()
    glm4.fit(np.float64(X.values[1:, use_cols]), np.float64(y.values[1:]))
    glm3 = glm4

    # pred = glm3.model.predict(X.values)
    # true = y.values

    print('SKGLM Done...')
    sk_int = glm3.intercept_
    sk_coef = glm3.coef_

    print('Starting PY GLM Net...')

    # glm = sglm.GLM('Normal', max_iter=10000, reg_lambda=0.00001, verbose=True, solver='batch-gradient')
    glm = sglm.GLM('Normal', max_iter=10000, reg_lambda=0.00001, verbose=True)
    glm.fit(np.float64(X.values[1:, use_cols]), np.float64(y.values[1:]))

    print('PY GLM Net Done...')

    # print(glm.intercept_, glm.coef_)



    sk_int = glm3.intercept_
    pgn_int = glm.intercept_

    sk_coef = glm3.coef_
    pgn_coef = glm.coef_

    int_vals = [('Intercept', sk_int, pgn_int)]
    coef_vals = [(column_names[_], sk_coef[_], pgn_coef[_]) for _ in range(len(use_cols))]

    with pd.option_context('max_rows', 1000):
        display(pd.DataFrame(int_vals + coef_vals, columns=['Col Name', 'SK Learn Coef', 'PY GLM Net Coef']))


    # Problem Columns...
    # > 'nTrial', 'iBlock'
    [
        'CuePenalty',
        'Penalty',
        'Select',
        'TO'
    ]
    [
        'switch',
        'selHigh'
    ]

    # Differences in coeffs -- come from sparse features
    # Newton CG taking too long to run -- comes from multicollinearity
    # 

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71] — Index(['CuePenalty', 'ENLPenalty', 'Select', 'Consumption', 'responseTime',
       'ENL', 'Cue', 'decision', 'switch', 'selR', 'selHigh', 'Reward', 'post',
       'CuePenalty_-2', 'ENLPenalty_-2', 'Select_-2', 'Consumption_-2',
       'responseTime_-2', 'ENL_-2', 'Cue_-2', 'decision_-2', 'switch_-2',
       'selR_-2', 'selHigh_-2', 'Reward_-2', 'post_-2', 'CuePenalty_-1',
       'ENLPenalty_-1', 'Select_-1', 'Consumption_-1', 'responseTime_-1',
       'ENL_-1', 'Cue_-1', 'decision_-1', 'switch_-1', 'selR_-1', 'selHigh_-1',
       'Reward_-1', 'post_-1', 'CuePenalty_1', 'ENLPenalty_1', 'Select_1',
       'Consumption_1', 'responseTime_1', 'ENL_1', 'Cue_1', 'decision_1',
       'switch_1', 'selR_1', 'selHigh_1', 'Reward_1', 'post_

Lambda: 0.0000


SKGLM Done...
Starting PY GLM Net...


	Parameter update tolerance. Converged in 3387 iterations


PY GLM Net Done...


,Col Name,SK Learn Coef,PY GLM Net Coef
0,Intercept,-0.020274,-0.020137
1,CuePenalty,-0.211815,-0.097288
2,ENLPenalty,-0.060191,-0.044937
3,Select,-0.882600,-0.845361
4,Consumption,-0.020996,-0.000000
5,responseTime,0.009937,-0.000000
6,ENL,0.092935,0.080779
7,Cue,-0.057069,-0.037235
8,decision,-0.131464,-0.139331
9,switch,-0.218030,-0.205507


In [12]:
crr = X[X.columns[use_cols]].dropna().corr()


In [23]:
X.iloc[:,:20].describe()

,nTrial,CuePenalty,ENLPenalty,Select,Consumption,responseTime,ENL,Cue,decision,switch,selR,selHigh,Reward,post,nTrial_-2,CuePenalty_-2,ENLPenalty_-2,Select_-2,Consumption_-2,responseTime_-2
count,234921.000000,234921.000000,234921.000000,234921.000000,234921.000000,234921.000000,234921.000000,234921.000000,234921.000000,234921.000000,234921.000000,234921.000000,234921.000000,234921.000000,234921.000000,234921.000000,234921.000000,234921.000000,234921.000000,234921.000000
mean,121.142367,0.000004,0.000047,0.001005,0.617825,0.046156,0.308521,0.017099,0.000009,-0.000817,0.000511,0.000656,0.314148,0.315214,121.144368,0.000004,0.000047,0.001005,0.617825,0.046156
std,68.145452,0.002063,0.006843,0.031679,0.485920,0.209823,0.461884,0.129642,0.031695,0.031685,0.031691,0.031689,0.720512,0.425441,68.145462,0.002063,0.006843,0.031679,0.485920,0.209823
min,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,62.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,62.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,121.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018520,121.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,180.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.881418,180.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,239.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.988810,239.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [15]:
with pd.option_context('max_rows',1000):
    display((crr-np.eye(len(crr))).max())

CuePenalty         0.015642
ENLPenalty         0.010245
Select             0.652354
Consumption        0.995745
responseTime       0.977182
ENL                0.995062
Cue                0.939974
decision           0.028462
switch             0.032797
selR               0.788073
selHigh            0.788073
Reward             0.998065
post               0.997481
CuePenalty_-2      0.015642
ENLPenalty_-2      0.010245
Select_-2          0.652354
Consumption_-2     0.995745
responseTime_-2    0.977182
ENL_-2             0.995062
Cue_-2             0.939974
decision_-2        0.008470
switch_-2          0.032797
selR_-2            0.788073
selHigh_-2         0.788073
Reward_-2          0.998065
post_-2            0.997481
CuePenalty_-1      0.015642
ENLPenalty_-1      0.010245
Select_-1          0.652354
Consumption_-1     0.995745
responseTime_-1    0.977182
ENL_-1             0.995062
Cue_-1             0.939974
decision_-1        0.028462
switch_-1          0.032797
selR_-1            0